In [1]:
import os 
import chromadb
from openai import OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "KEY"

In [3]:
openai_client = OpenAI()
chroma_client = chromadb.Client()

In [6]:
import time

def response_text(openai_resp):
    return openai_resp.choices[0].message.content

question = "북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?"

for _ in range(2):
    start_time = time.time()
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": question
            }
        ],
    )
    response = response_text(response)
    print(f"질문: {question}") 
    print(f"소요 시간: {(time.time() - start_time):.2f} ")
    print(f"답변: {response}\n")

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 1.39 
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 주로 가을부터 겨울까지이며, 보통 10월부터 다음해 3월까지 기간 동안 영향을 미칩니다.

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 1.64 
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 보통 봄과 가을에 발생하는데, 주로 3월부터 5월까지와 9월부터 11월까지 국내에 영향을 줍니다. 이 기간에는 갑자기 찬 바람이 불거나 기온이 급격히 떨어지는 현상이 나타날 수 있으니 주의해야 합니다.



In [ ]:
class OpenAICache: # 일치 캐시
    def __init__(self, openai_client):
        self.openai_client = openai_client
        self.cache = {}

    def generate(self, prompt):
        if prompt not in self.cache:
            response = self.openai_client.chat.completions.create(
                model = "gpt-3.5-turbo",
                messages = [
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
            )
            self.cache[prompt] = response_text(response)
        return self.cache[prompt]

        

In [11]:
openai_cache = OpenAICache(openai_client)

In [12]:
for _ in range(2):
    start_time = time.time()
    response = openai_cache.generate(question)
    print(f"질문: {question}") 
    print(f"소요 시간: {(time.time() - start_time):.2f} ")
    print(f"답변: {response}\n")

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 1.70 
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 주로 겨울철인 11월부터 다음 해 3월까지입니다. 이 기간 동안 한반도 일부 지역에서는 한파와 강풍 등으로 추운 날씨가 이어질 수 있습니다.

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 0.00 
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 주로 겨울철인 11월부터 다음 해 3월까지입니다. 이 기간 동안 한반도 일부 지역에서는 한파와 강풍 등으로 추운 날씨가 이어질 수 있습니다.



In [22]:
class OpenAICache:
    def __init__(self, openai_client, semantic_cache):
        self.openai_client = openai_client
        self.cache = {}
        self.semantic_cache = semantic_cache

    def generate(self, prompt):
        if prompt not in self.cache:
            similar_doc = self.semantic_cache.query(query_texts = [prompt], n_results = 1)
            if len(similar_doc["distances"][0]) > 0 and similar_doc["distances"][0][0] < 0.2:
                return similar_doc["metadatas"][0][0]["response"]
            else:
                response = self.openai_client.chat.completions.create(
                    model = "gpt-3.5-turbo",
                    messages = [
                        {
                            "role": "user",
                            "content": prompt
                        }
                    ],
                )
                self.cache[prompt] = response_text(response)
                self.semantic_cache.add(documents = [prompt],
                                        metadatas = [{"response": response_text(response)}], ids = [prompt])
        return self.cache[prompt]

In [ ]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

openai_ef = OpenAIEmbeddingFunction(
    api_key = os.environ["OPENAI_API_KEY"],
    model_name = "text-embedding-ada-002"
)

semantic_cache = chroma_client.create_collection(name="semantic_cache",
                                                 embedding_function = openai_ef,
                                                 metadata = {"hnsw:space": "cosine"})
openai_cache = OpenAICache(openai_client, semantic_cache)

In [25]:
questions = ["북태명양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?",
            "북태명양 기단과 오호츠크해 기단이 만나 국내에 머무르느 기간은?",
            "북태명양 기단과 오호츠크해 기단이 만나 한반도에 머무르는 기간은?",
            "국내에 북태평양 기단과 오호츠크해 기단이 함께 머무르는 기간은?"]

for question in questions:
    start_time = time.time()
    response = openai_cache.generate(question)
    print(f"질문: {question}") 
    print(f"소요 시간: {(time.time() - start_time):.2f} ")
    print(f"답변: {response}\n")

질문: 북태명양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 2.64 
답변: 북태명양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 겨울철인 12월부터 3월까지입니다. 이 기간 동안 새들은 대양을 건너 이동하는데 먹이를 찾기 위해 국내 해역에 머무르게 됩니다.

질문: 북태명양 기단과 오호츠크해 기단이 만나 국내에 머무르느 기간은?
소요 시간: 0.48 
답변: 북태명양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 겨울철인 12월부터 3월까지입니다. 이 기간 동안 새들은 대양을 건너 이동하는데 먹이를 찾기 위해 국내 해역에 머무르게 됩니다.

질문: 북태명양 기단과 오호츠크해 기단이 만나 한반도에 머무르는 기간은?
소요 시간: 0.59 
답변: 북태명양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 겨울철인 12월부터 3월까지입니다. 이 기간 동안 새들은 대양을 건너 이동하는데 먹이를 찾기 위해 국내 해역에 머무르게 됩니다.

질문: 국내에 북태평양 기단과 오호츠크해 기단이 함께 머무르는 기간은?
소요 시간: 0.58 
답변: 북태명양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 겨울철인 12월부터 3월까지입니다. 이 기간 동안 새들은 대양을 건너 이동하는데 먹이를 찾기 위해 국내 해역에 머무르게 됩니다.

